In [1]:
from pymongo import MongoClient,DESCENDING,ASCENDING
import re
import pandas as pd
import numpy as np
from city_funct import *

dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

mongodb://localhost/companies


In [2]:
companies_list=db.companies.find({},{"name":1,"number_of_employees":1,"founded_year":1,"total_money_raised":1,"offices":1,"category_code":1})
df= pd.DataFrame(companies_list)
df = df.explode("offices")
df


,_id,name,category_code,number_of_employees,founded_year,total_money_raised,offices
0,52cdef7c4bab8bd675297d8c,Zoho,software,1600.0,2005.0,$0,"{'description': 'Headquarters', 'address1': '4..."
1,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,600.0,1996.0,$0,"{'description': 'Headquarters', 'address1': '4..."
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,$39.8M,"{'description': '', 'address1': '710 - 2nd Ave..."
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,$39.8M,"{'description': '', 'address1': '270 Lafayette..."
3,52cdef7c4bab8bd675297d8d,Digg,news,60.0,2004.0,$45M,"{'description': None, 'address1': '135 Mississ..."
...,...,...,...,...,...,...,...
18796,52cdef7f4bab8bd67529c6f6,Adhunk,advertising,3.0,2009.0,$0,"{'description': 'Indian Office', 'address1': '..."
18797,52cdef7f4bab8bd67529c6f8,goBookmaker,web,NaN,NaN,$0,NaN
18798,52cdef7f4bab8bd67529c6f9,EnteGreat Solutions,software,NaN,NaN,$0,"{'description': '', 'address1': '', 'address2'..."
18799,52cdef7f4bab8bd67529c6ee,EnterSys Group,software,NaN,2000.0,$0,"{'description': '', 'address1': '13280 NW Free..."


In [3]:
def city_finder(x):
    finder=x.offices
    if type(finder) == dict:
        if "city" in finder:
            if finder["city"]:
                return finder["city"]
    else:
        return "NoOffice"

In [4]:
city = pd.DataFrame(df.apply(city_finder, axis=1))
city.columns=["city"]
city.head()

,city
0,Pleasanton
1,Pleasanton
2,Seattle
2,New York
3,San Francisco


In [5]:
df = pd.concat([df,city], axis=1)

In [6]:
def geopoint_office(x):
    finder = x.offices
    if type(finder) == dict:
        if 'latitude' in finder and 'longitude' in finder:
            if finder["latitude"] and finder["longitude"]:
                return ({
                    "type":"Point",
                    "coordinates":[float(finder["longitude"]),float(finder["latitude"])]
                },"success")
        return (None,"No office")

In [7]:
offices_set = df.apply(geopoint_office,axis=1,result_type="expand")
offices_set.columns = ["place","result"]
offices_set

,place,result
0,"{'type': 'Point', 'coordinates': [-121.904945,...",success
1,"{'type': 'Point', 'coordinates': [-121.904945,...",success
2,"{'type': 'Point', 'coordinates': [-122.333253,...",success
2,"{'type': 'Point', 'coordinates': [-73.9964312,...",success
3,"{'type': 'Point', 'coordinates': [-122.394523,...",success
...,...,...
18796,None,No office
18797,None,None
18798,"{'type': 'Point', 'coordinates': [-86.816068, ...",success
18799,"{'type': 'Point', 'coordinates': [-95.5045451,...",success


In [8]:
df=pd.concat([df,offices_set],axis=1)
df

,_id,name,category_code,number_of_employees,founded_year,total_money_raised,offices,city,place,result
0,52cdef7c4bab8bd675297d8c,Zoho,software,1600.0,2005.0,$0,"{'description': 'Headquarters', 'address1': '4...",Pleasanton,"{'type': 'Point', 'coordinates': [-121.904945,...",success
1,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,600.0,1996.0,$0,"{'description': 'Headquarters', 'address1': '4...",Pleasanton,"{'type': 'Point', 'coordinates': [-121.904945,...",success
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,$39.8M,"{'description': '', 'address1': '710 - 2nd Ave...",Seattle,"{'type': 'Point', 'coordinates': [-122.333253,...",success
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,$39.8M,"{'description': '', 'address1': '270 Lafayette...",New York,"{'type': 'Point', 'coordinates': [-73.9964312,...",success
3,52cdef7c4bab8bd675297d8d,Digg,news,60.0,2004.0,$45M,"{'description': None, 'address1': '135 Mississ...",San Francisco,"{'type': 'Point', 'coordinates': [-122.394523,...",success
...,...,...,...,...,...,...,...,...,...,...
18796,52cdef7f4bab8bd67529c6f6,Adhunk,advertising,3.0,2009.0,$0,"{'description': 'Indian Office', 'address1': '...",Bikaner,None,No office
18797,52cdef7f4bab8bd67529c6f8,goBookmaker,web,NaN,NaN,$0,NaN,NoOffice,None,None
18798,52cdef7f4bab8bd67529c6f9,EnteGreat Solutions,software,NaN,NaN,$0,"{'description': '', 'address1': '', 'address2'...",Birmingham,"{'type': 'Point', 'coordinates': [-86.816068, ...",success
18799,52cdef7f4bab8bd67529c6ee,EnterSys Group,software,NaN,2000.0,$0,"{'description': '', 'address1': '13280 NW Free...",Houston,"{'type': 'Point', 'coordinates': [-95.5045451,...",success


In [9]:
def coordinates_map(x):
    coor = x.place
    if type(coor)==dict:
        return {
        "latitude":float(coor["coordinates"][1]),
        "longitude":float(coor["coordinates"][0])
        }
    else:
        return("No coordinates")

In [10]:
coordinates = df.apply(coordinates_map,axis=1, result_type="expand")
coordinates

,latitude,longitude
0,37.6929,-121.905
1,37.6929,-121.905
2,47.6031,-122.333
2,40.7237,-73.9964
3,37.7647,-122.395
...,...,...
18796,No coordinates,No coordinates
18797,No coordinates,No coordinates
18798,33.5189,-86.8161
18799,29.8488,-95.5045


In [11]:
df = pd.concat([df,coordinates],axis=1)

In [12]:
df

,_id,name,category_code,number_of_employees,founded_year,total_money_raised,offices,city,place,result,latitude,longitude
0,52cdef7c4bab8bd675297d8c,Zoho,software,1600.0,2005.0,$0,"{'description': 'Headquarters', 'address1': '4...",Pleasanton,"{'type': 'Point', 'coordinates': [-121.904945,...",success,37.6929,-121.905
1,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,600.0,1996.0,$0,"{'description': 'Headquarters', 'address1': '4...",Pleasanton,"{'type': 'Point', 'coordinates': [-121.904945,...",success,37.6929,-121.905
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,$39.8M,"{'description': '', 'address1': '710 - 2nd Ave...",Seattle,"{'type': 'Point', 'coordinates': [-122.333253,...",success,47.6031,-122.333
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,$39.8M,"{'description': '', 'address1': '270 Lafayette...",New York,"{'type': 'Point', 'coordinates': [-73.9964312,...",success,40.7237,-73.9964
3,52cdef7c4bab8bd675297d8d,Digg,news,60.0,2004.0,$45M,"{'description': None, 'address1': '135 Mississ...",San Francisco,"{'type': 'Point', 'coordinates': [-122.394523,...",success,37.7647,-122.395
...,...,...,...,...,...,...,...,...,...,...,...,...
18796,52cdef7f4bab8bd67529c6f6,Adhunk,advertising,3.0,2009.0,$0,"{'description': 'Indian Office', 'address1': '...",Bikaner,None,No office,No coordinates,No coordinates
18797,52cdef7f4bab8bd67529c6f8,goBookmaker,web,NaN,NaN,$0,NaN,NoOffice,None,None,No coordinates,No coordinates
18798,52cdef7f4bab8bd67529c6f9,EnteGreat Solutions,software,NaN,NaN,$0,"{'description': '', 'address1': '', 'address2'...",Birmingham,"{'type': 'Point', 'coordinates': [-86.816068, ...",success,33.5189,-86.8161
18799,52cdef7f4bab8bd67529c6ee,EnterSys Group,software,NaN,2000.0,$0,"{'description': '', 'address1': '13280 NW Free...",Houston,"{'type': 'Point', 'coordinates': [-95.5045451,...",success,29.8488,-95.5045


In [13]:

df["total_money_raised"] = df["total_money_raised"].replace({'\$':''}, regex = True)
df["total_money_raised"] = df["total_money_raised"].replace({'\€':''}, regex = True)
df["total_money_raised"] = df["total_money_raised"].replace({'\£':''}, regex = True)

In [14]:
df['Last_Digit'] = df['total_money_raised'].str.strip().str[-1]
df['First_Digit'] = df['total_money_raised'].str.strip().str[0]


In [15]:
df

,_id,name,category_code,number_of_employees,founded_year,total_money_raised,offices,city,place,result,latitude,longitude,Last_Digit,First_Digit
0,52cdef7c4bab8bd675297d8c,Zoho,software,1600.0,2005.0,0,"{'description': 'Headquarters', 'address1': '4...",Pleasanton,"{'type': 'Point', 'coordinates': [-121.904945,...",success,37.6929,-121.905,0,0
1,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,600.0,1996.0,0,"{'description': 'Headquarters', 'address1': '4...",Pleasanton,"{'type': 'Point', 'coordinates': [-121.904945,...",success,37.6929,-121.905,0,0
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,39.8M,"{'description': '', 'address1': '710 - 2nd Ave...",Seattle,"{'type': 'Point', 'coordinates': [-122.333253,...",success,47.6031,-122.333,M,3
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,39.8M,"{'description': '', 'address1': '270 Lafayette...",New York,"{'type': 'Point', 'coordinates': [-73.9964312,...",success,40.7237,-73.9964,M,3
3,52cdef7c4bab8bd675297d8d,Digg,news,60.0,2004.0,45M,"{'description': None, 'address1': '135 Mississ...",San Francisco,"{'type': 'Point', 'coordinates': [-122.394523,...",success,37.7647,-122.395,M,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18796,52cdef7f4bab8bd67529c6f6,Adhunk,advertising,3.0,2009.0,0,"{'description': 'Indian Office', 'address1': '...",Bikaner,None,No office,No coordinates,No coordinates,0,0
18797,52cdef7f4bab8bd67529c6f8,goBookmaker,web,NaN,NaN,0,NaN,NoOffice,None,None,No coordinates,No coordinates,0,0
18798,52cdef7f4bab8bd67529c6f9,EnteGreat Solutions,software,NaN,NaN,0,"{'description': '', 'address1': '', 'address2'...",Birmingham,"{'type': 'Point', 'coordinates': [-86.816068, ...",success,33.5189,-86.8161,0,0
18799,52cdef7f4bab8bd67529c6ee,EnterSys Group,software,NaN,2000.0,0,"{'description': '', 'address1': '13280 NW Free...",Houston,"{'type': 'Point', 'coordinates': [-95.5045451,...",success,29.8488,-95.5045,0,0


In [16]:
cat= ['k','M','B']
df_clean= df[(df["Last_Digit"].isin(cat))]

In [17]:
df_clean["Last_Digit"].value_counts()

M    5363
k     855
B      17
Name: Last_Digit, dtype: int64

In [18]:
df_clean

,_id,name,category_code,number_of_employees,founded_year,total_money_raised,offices,city,place,result,latitude,longitude,Last_Digit,First_Digit
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,39.8M,"{'description': '', 'address1': '710 - 2nd Ave...",Seattle,"{'type': 'Point', 'coordinates': [-122.333253,...",success,47.6031,-122.333,M,3
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,39.8M,"{'description': '', 'address1': '270 Lafayette...",New York,"{'type': 'Point', 'coordinates': [-73.9964312,...",success,40.7237,-73.9964,M,3
3,52cdef7c4bab8bd675297d8d,Digg,news,60.0,2004.0,45M,"{'description': None, 'address1': '135 Mississ...",San Francisco,"{'type': 'Point', 'coordinates': [-122.394523,...",success,37.7647,-122.395,M,4
4,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,NaN,2005.0,800k,"{'description': '', 'address1': 'Suite 200', '...",Palo Alto,None,No office,No coordinates,No coordinates,k,8
7,52cdef7c4bab8bd675297d91,Geni,web,18.0,2006.0,16.5M,"{'description': 'Headquarters', 'address1': '9...",West Hollywood,"{'type': 'Point', 'coordinates': [-118.393064,...",success,34.0904,-118.393,M,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18776,52cdef7f4bab8bd67529c6e3,InSound Medical,biotech,NaN,NaN,11M,"{'description': '', 'address1': '39660 Eureka ...",Newark,"{'type': 'Point', 'coordinates': [-121.998513,...",success,37.512,-121.999,M,1
18777,52cdef7f4bab8bd67529c6e4,Getyoo,mobile,10.0,2009.0,1.13M,"{'description': 'HQ', 'address1': 'Rue des Tan...",Brussels,"{'type': 'Point', 'coordinates': [4.3464721, 5...",success,50.84,4.34647,M,1
18778,52cdef7f4bab8bd67529c6dd,Cameron Health,medical,NaN,2001.0,121M,"{'description': 'Corporate Office', 'address1'...",San Clemente,"{'type': 'Point', 'coordinates': [-117.607717,...",success,33.448,-117.608,M,1
18785,52cdef7f4bab8bd67529c6eb,DocASAP,web,7.0,2012.0,800k,"{'description': '', 'address1': '115 5th Ave',...",New York,None,No office,No coordinates,No coordinates,k,8


In [19]:
exclude =["1","2","3","4","5","6","7","8","9"]
df_clean= df_clean[(df_clean["First_Digit"].isin(exclude))]

In [20]:
df_clean

,_id,name,category_code,number_of_employees,founded_year,total_money_raised,offices,city,place,result,latitude,longitude,Last_Digit,First_Digit
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,39.8M,"{'description': '', 'address1': '710 - 2nd Ave...",Seattle,"{'type': 'Point', 'coordinates': [-122.333253,...",success,47.6031,-122.333,M,3
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,39.8M,"{'description': '', 'address1': '270 Lafayette...",New York,"{'type': 'Point', 'coordinates': [-73.9964312,...",success,40.7237,-73.9964,M,3
3,52cdef7c4bab8bd675297d8d,Digg,news,60.0,2004.0,45M,"{'description': None, 'address1': '135 Mississ...",San Francisco,"{'type': 'Point', 'coordinates': [-122.394523,...",success,37.7647,-122.395,M,4
4,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,NaN,2005.0,800k,"{'description': '', 'address1': 'Suite 200', '...",Palo Alto,None,No office,No coordinates,No coordinates,k,8
7,52cdef7c4bab8bd675297d91,Geni,web,18.0,2006.0,16.5M,"{'description': 'Headquarters', 'address1': '9...",West Hollywood,"{'type': 'Point', 'coordinates': [-118.393064,...",success,34.0904,-118.393,M,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18776,52cdef7f4bab8bd67529c6e3,InSound Medical,biotech,NaN,NaN,11M,"{'description': '', 'address1': '39660 Eureka ...",Newark,"{'type': 'Point', 'coordinates': [-121.998513,...",success,37.512,-121.999,M,1
18777,52cdef7f4bab8bd67529c6e4,Getyoo,mobile,10.0,2009.0,1.13M,"{'description': 'HQ', 'address1': 'Rue des Tan...",Brussels,"{'type': 'Point', 'coordinates': [4.3464721, 5...",success,50.84,4.34647,M,1
18778,52cdef7f4bab8bd67529c6dd,Cameron Health,medical,NaN,2001.0,121M,"{'description': 'Corporate Office', 'address1'...",San Clemente,"{'type': 'Point', 'coordinates': [-117.607717,...",success,33.448,-117.608,M,1
18785,52cdef7f4bab8bd67529c6eb,DocASAP,web,7.0,2012.0,800k,"{'description': '', 'address1': '115 5th Ave',...",New York,None,No office,No coordinates,No coordinates,k,8


In [21]:
df_clean["First_Digit"].value_counts()

1    1884
2    1145
3     804
5     627
4     519
6     397
7     313
8     288
9     224
Name: First_Digit, dtype: int64

In [22]:
multipliers = {'k':1000, 'M':1000000, 'B':1000000000}
count=0
for i in df_clean.total_money_raised:
    if i[-1].isdigit(): # check if no suffix
        print(i)
        df_clean["total_money_raised"].iloc[count] = int(string)
        count+=1
    else:
        mult = multipliers[i[-1]] # look up suffix to get multiplier
        df_clean["total_money_raised"].iloc[count] = int(float(i[:-1]) * mult)
        count+=1

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-22-eeda7c74aab7>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["total_money_raised"].iloc[count] = int(float(i[:-1]) * mult)


In [23]:
df_clean

,_id,name,category_code,number_of_employees,founded_year,total_money_raised,offices,city,place,result,latitude,longitude,Last_Digit,First_Digit
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,39800000,"{'description': '', 'address1': '710 - 2nd Ave...",Seattle,"{'type': 'Point', 'coordinates': [-122.333253,...",success,47.6031,-122.333,M,3
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,39800000,"{'description': '', 'address1': '270 Lafayette...",New York,"{'type': 'Point', 'coordinates': [-73.9964312,...",success,40.7237,-73.9964,M,3
3,52cdef7c4bab8bd675297d8d,Digg,news,60.0,2004.0,45000000,"{'description': None, 'address1': '135 Mississ...",San Francisco,"{'type': 'Point', 'coordinates': [-122.394523,...",success,37.7647,-122.395,M,4
4,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,NaN,2005.0,800000,"{'description': '', 'address1': 'Suite 200', '...",Palo Alto,None,No office,No coordinates,No coordinates,k,8
7,52cdef7c4bab8bd675297d91,Geni,web,18.0,2006.0,16500000,"{'description': 'Headquarters', 'address1': '9...",West Hollywood,"{'type': 'Point', 'coordinates': [-118.393064,...",success,34.0904,-118.393,M,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18776,52cdef7f4bab8bd67529c6e3,InSound Medical,biotech,NaN,NaN,11000000,"{'description': '', 'address1': '39660 Eureka ...",Newark,"{'type': 'Point', 'coordinates': [-121.998513,...",success,37.512,-121.999,M,1
18777,52cdef7f4bab8bd67529c6e4,Getyoo,mobile,10.0,2009.0,1130000,"{'description': 'HQ', 'address1': 'Rue des Tan...",Brussels,"{'type': 'Point', 'coordinates': [4.3464721, 5...",success,50.84,4.34647,M,1
18778,52cdef7f4bab8bd67529c6dd,Cameron Health,medical,NaN,2001.0,121000000,"{'description': 'Corporate Office', 'address1'...",San Clemente,"{'type': 'Point', 'coordinates': [-117.607717,...",success,33.448,-117.608,M,1
18785,52cdef7f4bab8bd67529c6eb,DocASAP,web,7.0,2012.0,800000,"{'description': '', 'address1': '115 5th Ave',...",New York,None,No office,No coordinates,No coordinates,k,8


In [24]:
limit=1000000
df_clean = df_clean[df_clean['total_money_raised']>=limit]
df_clean


,_id,name,category_code,number_of_employees,founded_year,total_money_raised,offices,city,place,result,latitude,longitude,Last_Digit,First_Digit
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,39800000,"{'description': '', 'address1': '710 - 2nd Ave...",Seattle,"{'type': 'Point', 'coordinates': [-122.333253,...",success,47.6031,-122.333,M,3
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,39800000,"{'description': '', 'address1': '270 Lafayette...",New York,"{'type': 'Point', 'coordinates': [-73.9964312,...",success,40.7237,-73.9964,M,3
3,52cdef7c4bab8bd675297d8d,Digg,news,60.0,2004.0,45000000,"{'description': None, 'address1': '135 Mississ...",San Francisco,"{'type': 'Point', 'coordinates': [-122.394523,...",success,37.7647,-122.395,M,4
7,52cdef7c4bab8bd675297d91,Geni,web,18.0,2006.0,16500000,"{'description': 'Headquarters', 'address1': '9...",West Hollywood,"{'type': 'Point', 'coordinates': [-118.393064,...",success,34.0904,-118.393,M,1
9,52cdef7c4bab8bd675297d95,StumbleUpon,web,NaN,2002.0,18500000,"{'description': '', 'address1': '', 'address2'...",San Francisco,"{'type': 'Point', 'coordinates': [-122.419204,...",success,37.7752,-122.419,M,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18775,52cdef7f4bab8bd67529c6e1,Purfresh,cleantech,NaN,1996.0,62700000,"{'description': 'HQ', 'address1': '1350 Willow...",Menlo Park,"{'type': 'Point', 'coordinates': [-122.151599,...",success,37.477,-122.152,M,6
18776,52cdef7f4bab8bd67529c6e3,InSound Medical,biotech,NaN,NaN,11000000,"{'description': '', 'address1': '39660 Eureka ...",Newark,"{'type': 'Point', 'coordinates': [-121.998513,...",success,37.512,-121.999,M,1
18777,52cdef7f4bab8bd67529c6e4,Getyoo,mobile,10.0,2009.0,1130000,"{'description': 'HQ', 'address1': 'Rue des Tan...",Brussels,"{'type': 'Point', 'coordinates': [4.3464721, 5...",success,50.84,4.34647,M,1
18778,52cdef7f4bab8bd67529c6dd,Cameron Health,medical,NaN,2001.0,121000000,"{'description': 'Corporate Office', 'address1'...",San Clemente,"{'type': 'Point', 'coordinates': [-117.607717,...",success,33.448,-117.608,M,1


In [25]:
df_clean["category_code"].value_counts()

software            928
web                 647
mobile              455
enterprise          436
advertising         385
games_video         329
biotech             257
cleantech           242
hardware            185
ecommerce           178
network_hosting     177
search              140
semiconductor       130
security            110
analytics            73
public_relations     72
other                63
finance              55
social               50
consulting           48
messaging            44
news                 40
music                36
medical              27
travel               25
education            24
health               21
manufacturing        20
photo_video          17
transportation       15
sports               11
legal                10
real_estate           8
fashion               7
automotive            7
design                5
nanotech              5
hospitality           5
nonprofit             3
Name: category_code, dtype: int64

In [26]:
industry=["software","web","mobile","games_video"]
df_clean= df_clean[(df_clean["category_code"].isin(industry))]
df_clean


,_id,name,category_code,number_of_employees,founded_year,total_money_raised,offices,city,place,result,latitude,longitude,Last_Digit,First_Digit
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,39800000,"{'description': '', 'address1': '710 - 2nd Ave...",Seattle,"{'type': 'Point', 'coordinates': [-122.333253,...",success,47.6031,-122.333,M,3
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,39800000,"{'description': '', 'address1': '270 Lafayette...",New York,"{'type': 'Point', 'coordinates': [-73.9964312,...",success,40.7237,-73.9964,M,3
7,52cdef7c4bab8bd675297d91,Geni,web,18.0,2006.0,16500000,"{'description': 'Headquarters', 'address1': '9...",West Hollywood,"{'type': 'Point', 'coordinates': [-118.393064,...",success,34.0904,-118.393,M,1
9,52cdef7c4bab8bd675297d95,StumbleUpon,web,NaN,2002.0,18500000,"{'description': '', 'address1': '', 'address2'...",San Francisco,"{'type': 'Point', 'coordinates': [-122.419204,...",success,37.7752,-122.419,M,1
9,52cdef7c4bab8bd675297d95,StumbleUpon,web,NaN,2002.0,18500000,"{'description': '', 'address1': '', 'address2'...",New York City,None,No office,No coordinates,No coordinates,M,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18721,52cdef7f4bab8bd67529c6ab,Vigilos,software,NaN,2000.0,7400000,"{'description': 'Headquarters', 'address1': '2...",Seattle,"{'type': 'Point', 'coordinates': [-122.3430581...",success,47.6113,-122.343,M,7
18724,52cdef7f4bab8bd67529c6ad,ANTs Software,software,3.0,1979.0,28300000,"{'description': 'Corporate Address', 'address1...",Dunwoody,"{'type': 'Point', 'coordinates': [-122.347654,...",success,37.5901,-122.348,M,2
18732,52cdef7f4bab8bd67529c6b6,Tiny Speck,games_video,45.0,2009.0,17200000,"{'description': '', 'address1': '163 Hastings ...",Vancouver,"{'type': 'Point', 'coordinates': [-123.109217,...",success,49.2825,-123.109,M,1
18766,52cdef7f4bab8bd67529c6d9,iGuiders,web,5.0,2008.0,1170000,"{'description': '', 'address1': '23811 Chagrin...",Beachwood,None,No office,No coordinates,No coordinates,M,1


In [27]:
place=["Seattle"]
df_city= df_clean[(df_clean["city"].isin(place))]
df_city



,_id,name,category_code,number_of_employees,founded_year,total_money_raised,offices,city,place,result,latitude,longitude,Last_Digit,First_Digit
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,39800000,"{'description': '', 'address1': '710 - 2nd Ave...",Seattle,"{'type': 'Point', 'coordinates': [-122.333253,...",success,47.6031,-122.333,M,3
126,52cdef7c4bab8bd675297e0b,Newsvine,web,8.0,2005.0,1250000,"{'description': None, 'address1': '101 Elliott...",Seattle,"{'type': 'Point', 'coordinates': [-122.358484,...",success,47.6186,-122.358,M,1
133,52cdef7c4bab8bd675297e11,iLike,games_video,28.0,2002.0,16500000,"{'description': None, 'address1': '1605 Boylst...",Seattle,"{'type': 'Point', 'coordinates': [-122.323408,...",success,47.6153,-122.323,M,1
227,52cdef7c4bab8bd675297e6d,PayScale,software,130.0,1999.0,31100000,"{'description': '', 'address1': '542 First Ave...",Seattle,"{'type': 'Point', 'coordinates': [-122.3276962...",success,47.5999,-122.328,M,3
238,52cdef7c4bab8bd675297e7b,BuddyTV,games_video,20.0,2005.0,10600000,"{'description': 'Seattle Headquarters', 'addre...",Seattle,"{'type': 'Point', 'coordinates': [-122.356756,...",success,47.6197,-122.357,M,1
378,52cdef7c4bab8bd675297f07,Shock Treatment Management,web,NaN,2004.0,4000000,"{'description': None, 'address1': '1601 2nd Av...",Seattle,"{'type': 'Point', 'coordinates': [-122.339859,...",success,47.6103,-122.34,M,4
477,52cdef7c4bab8bd675297f67,EyeJot,web,5.0,2007.0,1150000,"{'description': '', 'address1': '315 5th Ave S...",Seattle,"{'type': 'Point', 'coordinates': [-122.3276962...",success,47.5999,-122.328,M,1
550,52cdef7c4bab8bd675297fae,FlowPlay,games_video,30.0,2006.0,3970000,"{'description': '', 'address1': '114 Alaskan W...",Seattle,"{'type': 'Point', 'coordinates': [-122.335943,...",success,47.6015,-122.336,M,3
650,52cdef7c4bab8bd675298015,DocuSign,software,NaN,2003.0,116000000,"{'description': '', 'address1': '1301 Second A...",Seattle,"{'type': 'Point', 'coordinates': [-122.335668,...",success,47.603,-122.336,M,1
700,52cdef7c4bab8bd675298047,Treemo Labs,mobile,NaN,2005.0,2550000,"{'description': 'HQ', 'address1': '101 Yesler ...",Seattle,"{'type': 'Point', 'coordinates': [-122.347533,...",success,47.6207,-122.348,M,2


In [28]:
df_city["_id"] = df_city["_id"].apply(lambda e: str(e))
df_city.to_json("./input/df_city.json",orient="records")
df_city.to_csv("./input/df_city.csv")




<ipython-input-28-840b62655a30>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_city["_id"] = df_city["_id"].apply(lambda e: str(e))
